In [1]:
import statsmodels.api as sm
from my_libs import *

/Users/ken/anaconda3/envs/python2/lib/python2.7/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


TypeError: Required argument 'month' (pos 2) not found

**Regress the SPY and VIX index**

* Need to translate the result into np.array
* Need to change type to float

In [24]:
spy = get_price_data(["SPY"],method='day',back_day=30).dropna().Return.values.astype(float)
spy_ = spy*30

All price data of Close is actually Adj Close
Connection Successful
Finished SPY


Constructed a model of vix = intercept + b0 \* spy + b1 \* spy * 30

In [25]:
#ip = pd.DataFrame({"spy":spy,"spy_":spy_})
ip = pd.DataFrame({"spy":spy})
dp = get_price_data(["VIXY"],method='day',back_day=30).dropna().Return.values.astype(float)

All price data of Close is actually Adj Close
Connection Successful
Finished VIXY


In [26]:
ip = sm.add_constant(ip)

In [27]:
sm.OLS(dp,ip).fit().summary()

/home/ken/.local/lib/python2.7/site-packages/scipy/stats/stats.py:1416: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.797
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     66.66
Date:                Sun, 04 Aug 2019   Prob (F-statistic):           2.76e-07
Time:                        22:28:39   Log-Likelihood:                 56.059
No. Observations:                  19   AIC:                            -108.1
Df Residuals:                      17   BIC:                            -106.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0010      0.003     -0.328      0.747      -0.008       0.006
spy           -4.6318      0.567     -8.165      0.000      -5.829      -3.435
==============================================================================
Omnibus:                        2.155   Durbin-Watson:                   2.272
Prob(Omnibus):                  0.341   Jarque-Bera (JB):                0.687
Skew:                           0.332   Prob(JB):                        0.709
Kurtosis:                       3.653   Cond. No.                         185.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
 def hedge(self,method = "self_minute",hedge_int = "VIXY",target_beta =0.0):
        hedge_int_beta =beta([hedge_int],interval ="minute",robinhood=self)[0].iloc[0][0]
        hedge_int_price = self.get_ask_price(hedge_int)
        stocks, quantity =self.get_my_positions()
        position_beta = self.get_protfolio(stocks, quantity,interval = method)[1]
        position_value = float(self.my_trader.portfolios()["last_core_equity"])
        shares = np.floor(position_value*(target_beta-position_beta)/hedge_int_beta/hedge_int_price)
        clear_output()
        return ("To headge the position beta to " + str(target_beta) + " please trade " + str(shares) + " share of " +str(hedge_int) + "<br>" + "Total Beta is {:1.4f}".format(position_beta))

-----

## Ranking Model

In [4]:
sql = '''
select * from all_symbol.ScreenerModel_TOS

'''

mongod = mongo()

rank = mongod.conn.get_data(sql)

Connection Successful


In [5]:
rank.columns

Index([u'Ticker', u'Sector', u'Industry', u'Refresh_Date', u'P/Cash',
       u'Analyst Recom', u'P/S', u'Total Debt/Equity', u'P/Free Cash Flow',
       u'P/E', u'Insider Ownership', u'Gross Margin', u'Current Ratio',
       u'Sales growth quarter over quarter', u'Profit Margin', u'Quick Ratio',
       u'Performance (Week)', u'Institutional Ownership', u'EPS (ttm)',
       u'Operating Margin'],
      dtype='object')

In [6]:
rank_model = rank.dropna(subset=["Performance (Week)","P/S","EPS (ttm)","P/E","Institutional Ownership"])

In [7]:
dp = rank_model["Performance (Week)"]

In [8]:
ip = rank_model[["P/S","EPS (ttm)","Institutional Ownership"]]

In [9]:
ip = ip.join(pd.get_dummies(rank_model.Sector))

In [10]:
ip = sm.add_constant(ip)
mo = sm.OLS(dp,ip).fit()

/home/ken/.local/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [11]:
mo.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     Performance (Week)   R-squared:                       0.670
Model:                            OLS   Adj. R-squared:                  0.669
Method:                 Least Squares   F-statistic:                     408.6
Date:                Mon, 15 Jun 2020   Prob (F-statistic):               0.00
Time:                        08:07:59   Log-Likelihood:                -16704.
No. Observations:                2628   AIC:                         3.344e+04
Df Residuals:                    2614   BIC:                         3.352e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                     209.4780      4.174     50.182      0.000     201.293     217.663
P/S                        -0.3109      0.018    -17.551      0.000      -0.346      -0.276
EPS (ttm)                  -0.0893      0.017     -5.189      0.000      -0.123      -0.056
Institutional Ownership    -0.3609      0.019    -19.202      0.000      -0.398      -0.324
Basic Materials           -93.1370     11.978     -7.776      0.000    -116.624     -69.650
Communication Services   -104.1105     12.786     -8.142      0.000    -129.183     -79.038
Consumer Cyclical          26.7874      9.014      2.972      0.003       9.112      44.463
Consumer Defensive        -95.7736     12.168     -7.871      0.000    -119.634     -71.913
Energy                    -85.2871     11.541     -7.390      0.000    -107.918     -62.657
Financial                 618.2170      9.652     64.054      0.000     599.292     637.143
Healthcare                -24.0887      9.934     -2.425      0.015     -43.568      -4.609
Industrials               127.0298      7.683     16.533      0.000     111.964     142.096
Real Estate               -63.5287     11.003     -5.774      0.000     -85.103     -41.954
Technology                 40.0252      8.420      4.754      0.000      23.515      56.536
Utilities                -136.6557     14.980     -9.122      0.000    -166.030    -107.282
==============================================================================
Omnibus:                      148.302   Durbin-Watson:                   2.015
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              594.434
Skew:                          -0.047   Prob(JB):                    8.32e-130
Kurtosis:                       5.328   Cond. No.                     6.40e+18
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.86e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [12]:
mo.params

const                      209.4780
P/S                         -0.3109
EPS (ttm)                   -0.0893
Institutional Ownership     -0.3609
Basic Materials            -93.1370
Communication Services    -104.1105
Consumer Cyclical           26.7874
Consumer Defensive         -95.7736
Energy                     -85.2871
Financial                  618.2170
Healthcare                 -24.0887
Industrials                127.0298
Real Estate                -63.5287
Technology                  40.0252
Utilities                 -136.6557
dtype: float64

In [13]:
rank["Model_Performance(Week)"]=mo.params["const"]+rank["P/S"]*mo.params["P/S"]+rank["EPS (ttm)"]*mo.params["EPS (ttm)"] + rank["Institutional Ownership"]*mo.params["Institutional Ownership"]

In [14]:
for i in rank.index:
    rank.loc[i,"Model_Performance(Week)"] = rank.loc[i,"Model_Performance(Week)"]+mo.params[rank.loc[i,"Sector"]]

In [15]:
rank[["Sector","Ticker","Performance (Week)","Model_Performance(Week)"]].sort_values("Performance (Week)").to_excel("test.xlsx")

In [173]:
rank[rank.Ticker == "PVAC"]

,Ticker,Sector,Industry,Refresh_Date,P/Cash,Analyst Recom,P/S,Total Debt/Equity,P/Free Cash Flow,P/E,Insider Ownership,Gross Margin,Current Ratio,Sales growth quarter over quarter,Profit Margin,Quick Ratio,Performance (Week),Institutional Ownership,EPS (ttm),Operating Margin,Model_Performance(Week)
2199,PVAC,Energy,Oil & Gas E&P,2020-06-14 21:31:03,16.0000,62.5000,24.0000,83.0000,nan,2.5000,52.5000,90.0000,85.0000,51.0000,112.0000,92.0000,1.0000,133.0000,143.0000,94.0000,-53.6260


In [21]:
def fwd_price(ticker,spot0 = None,riskless_rate=0,mat = 1,val_num_steps = 30,val_num_paths = 50000,yield_ = 0 ):
    if spot0 is None:
        spot0=realtimequote(ticker).price.values[0]

    # vol = get_price_data(["SPY"],method = "self_minute", robinhood = robingateway()).Return.std()
    option = option_trading(robingateway(),ticker,"call","buy",0)
    vol = float(option.current_Option()["implied_volatility"])

#     vol = get_price_data([ticker],method = "self_minute", robinhood = robingateway()).Return.std()
    
    val_random_seed =  int(time.time())
    np.random.seed(val_random_seed)
    dt = float(mat)/ val_num_steps
    sqrt_dt = dt ** 0.5
    
    drift = (riskless_rate - yield_ - (vol ** 2) / 2) * dt
    result = np.empty([val_num_paths, val_num_steps])
    for path_num in xrange(val_num_paths):
        result1 = np.empty([val_num_steps])
        spot = spot0
        result1[0] = spot
        for step_num in xrange(1, val_num_steps):
            spot *= exp(drift + vol * np.random.normal() * sqrt_dt)
            if hasattr(yield_, '__call__'):
                div_t1, div_t2 = dt * step_num, dt * (step_num + 1)
                div = yield_(div_t1, div_t2)
                if is_number(div): spot -= div
            result1[step_num] = spot

        result[path_num] = result1
    return pd.DataFrame(result).mean().iloc[-1]
    

In [22]:
def fwd_return(ticker,riskless_rate=0,mat = 1,val_num_steps = 30,val_num_paths = 50000):
    spot = get_price_data([ticker],method = "self_minute", robinhood = robingateway()).Close.iloc[-1]

    fprice = fwd_price(ticker,spot,riskless_rate,mat,val_num_steps,val_num_paths)
    print (fprice)
        
    return np.log(fprice/spot)

In [16]:
fwd_price("AAL")

Ascending
2020-06-26
Ascending
Going ATP
current option:
 Symbol:AAL strike:16.0, expiration:2020-06-26, type:call
current option:
 Symbol:AAL strike:16.0, expiration:2020-06-26, type:call


15.899537290597967

In [17]:
price["Fwd_Return"].iloc[-1]= fwd_return("AAL")

All price data of Close is actually Adj Close
Connection Successful
('Finished', 'AAL')
Ascending
2020-06-26
Ascending
Going ATP
current option:
 Symbol:AAL strike:16.0, expiration:2020-06-26, type:call
current option:
 Symbol:AAL strike:16.0, expiration:2020-06-26, type:call
15.883571604744503


/home/ken/.local/lib/python2.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
i = "SPY"

In [16]:
get_price_data([i], method="day",robinhood=robingateway(),back_day=200 )

All price data of Close is actually Adj Close
('Finished', u'SPY')


,adjClose,change,changeOverTime,changePercent,Close,TimeStamp,High,label,Low,Open,unadjustedVolume,Volume,vwap,Ticker,Return,Fwd_Return
136,308.6800,-0.2100,-0.0007,-0.0670,312.0200,2019-12-05,312.2500,"December 05, 19",310.5800,312.2300,"40,709,000.0000","40,709,000.0000",311.6167,SPY,nan,0.0091
135,311.5000,0.7500,0.0024,0.2390,314.8700,2019-12-06,315.3100,"December 06, 19",314.1100,314.1200,"48,927,000.0000","48,927,000.0000",314.7633,SPY,0.0091,-0.0031
134,310.5200,-0.5600,-0.0018,-0.1780,313.8800,2019-12-09,315.1800,"December 09, 19",313.8000,314.4400,"34,838,500.0000","34,838,500.0000",314.2867,SPY,-0.0032,-0.0011
133,310.1700,-0.2900,-0.0009,-0.0920,313.5300,2019-12-10,314.5500,"December 10, 19",312.8100,313.8200,"52,649,800.0000","52,649,800.0000",313.6300,SPY,-0.0011,0.0028
132,311.0500,0.3900,0.0012,0.1240,314.4200,2019-12-11,314.7000,"December 11, 19",313.4400,314.0300,"53,429,100.0000","53,429,100.0000",314.1867,SPY,0.0028,0.0086
131,313.7300,2.7000,0.0086,0.8590,317.1300,2019-12-12,317.9900,"December 12, 19",314.1700,314.4300,"96,389,600.0000","96,389,600.0000",316.4300,SPY,0.0085,0.0006
130,313.9200,0.4500,0.0014,0.1420,317.3200,2019-12-13,318.6700,"December 13, 19",316.0200,316.8700,"81,503,900.0000","81,503,900.0000",317.3367,SPY,0.0006,0.0068
129,316.0800,0.2800,0.0009,0.0880,319.5000,2019-12-16,320.1500,"December 16, 19",317.2500,319.2200,"82,749,700.0000","82,749,700.0000",318.9667,SPY,0.0068,0.0002
128,316.1500,-0.3500,-0.0011,-0.1090,319.5700,2019-12-17,320.2500,"December 17, 19",319.4800,319.9200,"61,097,700.0000","61,097,700.0000",319.7667,SPY,0.0002,0.0001
127,316.1700,-0.4100,-0.0013,-0.1280,319.5900,2019-12-18,320.2500,"December 18, 19",319.5300,320.0000,"48,133,000.0000","48,133,000.0000",319.7900,SPY,0.0001,0.0041


In [17]:
 mongod = mongo("all_symbol")

# candidates = pd.DataFrame(mongod.db["momentum"].find()).sort_values("End_Value", ascending=False)    

candidates = mongod.conn.get_data('''

select * from all_symbol.momentum

where Refresh_Date = (select max(Refresh_Date) from all_symbol.momentum)

order by end_value desc

''')  

Connection Successful


In [12]:
fwd_price("^VIX",mat=3,val_num_steps=42)

Ascending
All price data of Close is actually Adj Close
('Finished', u'^VIX')


33.10881949903828

In [8]:
realtimequote("^VIX").price

0   33.3100
Name: price, dtype: float64

In [18]:
vix = vix_dayroll_trade(robingateway())

In [29]:
my_future = future()
##Changed to use faward price 6/22/2020
day_roll = ((my_future.get_future_price(datetime.now().year,datetime.now().month) - realtimequote("^VIX").price)/my_future.get_historic_data(my_future.get_future_expiration(datetime.now().year,datetime.now().month)).Close.std())/30



VXM20
The previous symbol not available, changed to: VXN20
VXM20
The previous symbol not available, changed to: VXN20


In [30]:
day_roll.values[0]

-0.00045382855837579523

In [19]:
vix.vix_dayroll()

VXM20
The previous symbol not available, changed to: VXN20
Ascending
All price data of Close is actually Adj Close
('Finished', u'^VIX')
VXM20
The previous symbol not available, changed to: VXN20


AttributeError: 'numpy.float64' object has no attribute 'values'

In [ ]:
val_num_steps = num_steps or DEFAULT_BINOMIAL_TREE_NUM_STEPS
val_cache = {}

dt = mat/ val_num_steps
up_fact = exp(self.vol * (dt ** 0.5))
down_fact = 1.0 / up_fact
cont_yield = 0 if hasattr(self.yield_, '__call__') else self.yield_
prob_up = (exp((self.riskless_rate - cont_yield) * dt) - down_fact) / (up_fact - down_fact)

def divs_pv(n):
    if not hasattr(self.yield_, '__call__'): return 0
    pv_divs = 0
    for step_num in xrange(n, val_num_steps):
        div_t1, div_t2 = dt * step_num, dt * (step_num + 1)
        div = self.yield_(div_t1, div_t2)
        if is_number(div):
            mid_div_t = 0.5 * (div_t1 + div_t2)
            pv_divs += exp(-self.riskless_rate * mid_div_t) * div
    return pv_divs

bin_spot0 = self.spot0 - divs_pv(0)

def spot_price(n, num_ups, num_downs):
    return (bin_spot0 + divs_pv(n)) * (up_fact ** (num_ups - num_downs))

def node_value(n, num_ups, num_downs):
    value_cache_key = (n, num_ups, num_downs)
    if value_cache_key not in val_cache:
        spot = spot_price(n, num_ups, num_downs)
        if self.opt_type == OptionType.CALL:
            exer_profit = max(0, spot - self.strike)
        else:  # opt_type == OptionType.PUT:
            exer_profit = max(0, self.strike - spot)

        if n >= val_num_steps:
            val = exer_profit
        else:
            fv = prob_up * node_value(n + 1, num_ups + 1, num_downs) + \
                (1 - prob_up) * node_value(n + 1, num_ups, num_downs + 1)
            pv = exp(-self.riskless_rate * dt) * fv
            if self.exer_type == OptionExerciseType.AMERICAN:
                val = max(pv, exer_profit)
            else:  # exer_type == OptionExerciseType.EUROPEAN
                val = pv

        val_cache[value_cache_key] = val
    return val_cache[value_cache_key]

In [2]:
import arch

In [10]:
print arch.arch_model.func_doc


    Convenience function to simplify initialization of ARCH models

    Parameters
    ----------
    y : {ndarray, Series, None}
        The dependent variable
    x : {np.array, DataFrame}, optional
        Exogenous regressors.  Ignored if model does not permit exogenous
        regressors.
    mean : str, optional
        Name of the mean model.  Currently supported options are: 'Constant',
        'Zero', 'ARX' and  'HARX'
    lags : int or list (int), optional
        Either a scalar integer value indicating lag length or a list of
        integers specifying lag locations.
    vol : str, optional
        Name of the volatility model.  Currently supported options are:
        'GARCH' (default), 'ARCH', 'EGARCH', 'FIARCH' and 'HARCH'
    p : int, optional
        Lag order of the symmetric innovation
    o : int, optional
        Lag order of the asymmetric innovation
    q : int, optional
        Lag order of lagged volatility or equivalent
    power : float, optional
        Po

In [3]:
robinhood = robingateway()

In [4]:
 mongod = mongo("all_symbol")

# all_ticker = pd.DataFrame(mongod.db["cantrade"].find()).Ticker.tolist()
result = pd.DataFrame()
all_ticker = mongod.conn.get_data("select * from all_symbol.cantrade").Ticker.tolist()

for i in all_ticker:
    try:
        print (i)
        first = momentum_best_day(i,robinhood=robinhood,method = "self_minute").iloc[0]
        end_value = momentum_backtest(i,first["Lookback"],first["Holding"],robinhood=robinhood,method = "self_minute")
        temp = pd.DataFrame([(i,end_value, first["Lookback"],first["Holding"])],columns= ["Ticker", "End_Value","Lookback","Holding"])
        temp["Refresh_Date"] = datetime.today().date()
        result = result.append(temp)
    except Exception as e:
        print (e)
#         send_email("screen momentum trade error\n" + str(e))
  

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ken/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/ken/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/ken/.local/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/usr/lib/python2.7/inspect.py", line 1058, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/usr/lib/python2.7/inspect.py", line 1018, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/usr/lib/python2.7/inspect.py", line 453, in getsourcefile
    if hasattr(getmodule(object, filename), '__loader__'):
  File "/usr/lib/python2.7/inspect.py", line 496, in getmodule
    f = getabsf

IndexError: string index out of range

In [2]:
mongod = mongo()

Connection Successful


In [ ]:
mongod.update_db_new_day()

AAPL
All price data of Close is actually Adj Close
('Finished', u'AAPL')
Probably not exist
If no error message, task completed
successed AAPL
BAC
All price data of Close is actually Adj Close
('Finished', u'BAC')
Probably not exist
If no error message, task completed
successed BAC
AMZN
All price data of Close is actually Adj Close
('Finished', u'AMZN')
Probably not exist
If no error message, task completed
successed AMZN
T
All price data of Close is actually Adj Close
('Finished', u'T')
Probably not exist
If no error message, task completed
successed T
GOOG
All price data of Close is actually Adj Close
('Finished', u'GOOG')
Probably not exist
If no error message, task completed
successed GOOG
MO
All price data of Close is actually Adj Close
('Finished', u'MO')
Probably not exist
If no error message, task completed
successed MO
DAL
All price data of Close is actually Adj Close
('Finished', u'DAL')
Probably not exist
If no error message, task completed
successed DAL
AA
All price data of

('Finished', u'AES')
Probably not exist
If no error message, task completed
successed AES
BUD
All price data of Close is actually Adj Close
('Finished', u'BUD')
Probably not exist
If no error message, task completed
successed BUD
ARR
All price data of Close is actually Adj Close
('Finished', u'ARR')
Probably not exist
If no error message, task completed
successed ARR
BDX
All price data of Close is actually Adj Close
('Finished', u'BDX')
Probably not exist
If no error message, task completed
successed BDX
AB
All price data of Close is actually Adj Close
('Finished', u'AB')
Probably not exist
If no error message, task completed
successed AB
CS
All price data of Close is actually Adj Close
('Finished', u'CS')
Probably not exist
If no error message, task completed
successed CS
AFL
All price data of Close is actually Adj Close
('Finished', u'AFL')
Probably not exist
If no error message, task completed
successed AFL
ADI
All price data of Close is actually Adj Close
('Finished', u'ADI')
Proba

('Finished', u'APH')
Probably not exist
If no error message, task completed
successed APH
AMX
All price data of Close is actually Adj Close
('Finished', u'AMX')
Probably not exist
If no error message, task completed
successed AMX
JKHY
All price data of Close is actually Adj Close
('Finished', u'JKHY')
Probably not exist
If no error message, task completed
successed JKHY
AVAV
All price data of Close is actually Adj Close
('Finished', u'AVAV')
Probably not exist
If no error message, task completed
successed AVAV
AMKR
All price data of Close is actually Adj Close
('Finished', u'AMKR')
Probably not exist
If no error message, task completed
successed AMKR
ERC
All price data of Close is actually Adj Close
('Finished', u'ERC')
Probably not exist
If no error message, task completed
successed ERC
AEIS
All price data of Close is actually Adj Close
('Finished', u'AEIS')
Probably not exist
If no error message, task completed
successed AEIS
Y
All price data of Close is actually Adj Close
('Finished

('Finished', u'AAWW')
Probably not exist
If no error message, task completed
successed AAWW
WMS
All price data of Close is actually Adj Close
('Finished', u'WMS')
Probably not exist
If no error message, task completed
successed WMS
PAG
All price data of Close is actually Adj Close
('Finished', u'PAG')
Probably not exist
If no error message, task completed
successed PAG
MDRX
All price data of Close is actually Adj Close
('Finished', u'MDRX')
Probably not exist
If no error message, task completed
successed MDRX
KLIC
All price data of Close is actually Adj Close
('Finished', u'KLIC')
Probably not exist
If no error message, task completed
successed KLIC
ETH
All price data of Close is actually Adj Close
('Finished', u'ETH')
Probably not exist
If no error message, task completed
successed ETH
AZPN
All price data of Close is actually Adj Close
('Finished', u'AZPN')
Probably not exist
If no error message, task completed
successed AZPN
AXE
All price data of Close is actually Adj Close
('Finishe

('Finished', u'TA')
Probably not exist
If no error message, task completed
successed TA
RPAI
All price data of Close is actually Adj Close
('Finished', u'RPAI')
Probably not exist
If no error message, task completed
successed RPAI
MANH
All price data of Close is actually Adj Close
('Finished', u'MANH')
Probably not exist
If no error message, task completed
successed MANH
LAMR
All price data of Close is actually Adj Close
('Finished', u'LAMR')
Probably not exist
If no error message, task completed
successed LAMR
KYN
All price data of Close is actually Adj Close
('Finished', u'KYN')
Probably not exist
If no error message, task completed
successed KYN
ATRO
All price data of Close is actually Adj Close
('Finished', u'ATRO')
Probably not exist
If no error message, task completed
successed ATRO
ATNI
All price data of Close is actually Adj Close
('Finished', u'ATNI')
Probably not exist
If no error message, task completed
successed ATNI
APEI
All price data of Close is actually Adj Close
('Fini

...

Probably not exist
If no error message, task completed
successed DQ
CWST
All price data of Close is actually Adj Close
('Finished', u'CWST')
Probably not exist
If no error message, task completed
successed CWST
CVTI
All price data of Close is actually Adj Close
('Finished', u'CVTI')
Probably not exist
If no error message, task completed
successed CVTI
CRAI
All price data of Close is actually Adj Close
('Finished', u'CRAI')
Probably not exist
If no error message, task completed
successed CRAI
CNCE
All price data of Close is actually Adj Close
('Finished', u'CNCE')
Probably not exist
If no error message, task completed
successed CNCE
CLS
All price data of Close is actually Adj Close
('Finished', u'CLS')
Probably not exist
If no error message, task completed
successed CLS
CIO
All price data of Close is actually Adj Close
('Finished', u'CIO')
Probably not exist
If no error message, task completed
successed CIO
CIA
All price data of Close is actually Adj Close
('Finished', u'CIA')
Probably 

('Finished', u'KOF')
Probably not exist
If no error message, task completed
successed KOF
KMPR
All price data of Close is actually Adj Close
('Finished', u'KMPR')
Probably not exist
If no error message, task completed
successed KMPR
IRMD
All price data of Close is actually Adj Close
('Finished', u'IRMD')
Probably not exist
If no error message, task completed
successed IRMD
IGR
All price data of Close is actually Adj Close
('Finished', u'IGR')
Probably not exist
If no error message, task completed
successed IGR
HURC
All price data of Close is actually Adj Close
('Finished', u'HURC')
Probably not exist
If no error message, task completed
successed HURC
GVA
All price data of Close is actually Adj Close
('Finished', u'GVA')
Probably not exist
If no error message, task completed
successed GVA
GTS
All price data of Close is actually Adj Close
('Finished', u'GTS')
Probably not exist
If no error message, task completed
successed GTS
GAIN
All price data of Close is actually Adj Close
('Finished

('Finished', u'CEMI')
Probably not exist
If no error message, task completed
successed CEMI
CDXS
All price data of Close is actually Adj Close
('Finished', u'CDXS')
Probably not exist
If no error message, task completed
successed CDXS
UTF
All price data of Close is actually Adj Close
('Finished', u'UTF')
Probably not exist
If no error message, task completed
successed UTF
SIM
All price data of Close is actually Adj Close
('Finished', u'SIM')
Probably not exist
If no error message, task completed
successed SIM
SFNC
All price data of Close is actually Adj Close
('Finished', u'SFNC')
Probably not exist
If no error message, task completed
successed SFNC
SEIC
All price data of Close is actually Adj Close
('Finished', u'SEIC')
Probably not exist
If no error message, task completed
successed SEIC
PRK
All price data of Close is actually Adj Close
('Finished', u'PRK')
Probably not exist
If no error message, task completed
successed PRK
OFED
All price data of Close is actually Adj Close
('Finish

('Finished', u'JJSF')
Probably not exist
If no error message, task completed
successed JJSF
IIN
All price data of Close is actually Adj Close
('Finished', u'IIN')
Probably not exist
If no error message, task completed
successed IIN
HCCI
All price data of Close is actually Adj Close
('Finished', u'HCCI')
Probably not exist
If no error message, task completed
successed HCCI
HAE
All price data of Close is actually Adj Close
('Finished', u'HAE')
Probably not exist
If no error message, task completed
successed HAE
FUNC
All price data of Close is actually Adj Close
('Finished', u'FUNC')
Probably not exist
If no error message, task completed
successed FUNC
FSV
All price data of Close is actually Adj Close
('Finished', u'FSV')
Probably not exist
If no error message, task completed
successed FSV
FOXF
All price data of Close is actually Adj Close
('Finished', u'FOXF')
Probably not exist
If no error message, task completed
successed FOXF
FLIC
All price data of Close is actually Adj Close
('Finish

('Finished', u'DLB')
Probably not exist
If no error message, task completed
successed DLB
ODFL
All price data of Close is actually Adj Close
('Finished', u'ODFL')
Probably not exist
If no error message, task completed
successed ODFL
DK
All price data of Close is actually Adj Close
('Finished', u'DK')


In [14]:
get_price_data([i], method = "intraday",start_date=start_date,end_date=end_date)

All price data of Close is actually Adj Close
('Finished', u'TSQ')


,Close,TimeStamp,High,Low,Open,Volume,Ticker,Time,Return,Fwd_Return
199,4.3100,2020-06-03 14:00:00,4.3100,4.3100,4.3100,18838,TSQ,140000,nan,0.0092
198,4.3500,2020-06-03 14:30:00,4.3512,4.3100,4.3100,20146,TSQ,143000,0.0092,-0.0035
197,4.3350,2020-06-03 15:00:00,4.3700,4.3350,4.3500,26341,TSQ,150000,-0.0035,0.0058
196,4.3600,2020-06-03 15:30:00,4.3800,4.3350,4.3350,27908,TSQ,153000,0.0057,-0.0115
195,4.3100,2020-06-03 16:00:00,4.3600,4.3100,4.3600,31108,TSQ,160000,-0.0116,0.0092
194,4.3500,2020-06-04 09:30:00,4.3500,4.3500,4.3500,31457,TSQ,93000,0.0092,0.0000
193,4.3500,2020-06-04 10:00:00,4.3500,4.3450,4.3450,2780,TSQ,100000,0.0000,0.0089
192,4.3891,2020-06-04 10:30:00,4.3891,4.3500,4.3500,4029,TSQ,103000,0.0089,0.0250
191,4.5000,2020-06-04 11:00:00,4.5000,4.5000,4.5000,12182,TSQ,110000,0.0246,-0.0045
190,4.4800,2020-06-04 11:30:00,4.6000,4.4800,4.5000,37198,TSQ,113000,-0.0045,0.0000


In [10]:
i = 'WTR'
db_name = "stocks_daily"

In [11]:
mongod = mongo()
temptemp = pd.DataFrame()
quote = get_price_data([i], method = "realtimeday",back_day = 1000)
#                     quote.TimeStamp= quote.TimeStamp.apply(lambda x: \
#                                 datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ"))
if len(quote) == 0:
    print ("No data from quote")
    
try:
    temp = self.conn.get_data("select * from `%s`.`%s`"%(db_name,i))

    self.conn.conn.cursor().execute("drop table `%s`.`%s`"%(db_name,i))
    self.conn.conn.commit()
    print ("Table Dropped")
except:
    temp = pd.DataFrame()
    print ("Probably not exist")
    pass

temp = temp.append(quote).drop_duplicates(subset=["TimeStamp"])
mongod.conn.to_sql(temp,db_name,i,if_exists="append")

Connection Successful
All price data of Close is actually Adj Close
('Finished', u'WTR')
Probably not exist
If no error message, task completed


In [3]:
send_email("error occored in minute data updating")

NameError: global name 'readgateway' is not defined